In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('lin_reg').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training=spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [5]:
training.show(5)

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 5 rows



In [6]:
lin_reg=LinearRegression(featuresCol='features',labelCol='label',predictionCol='prediction')

In [7]:
model=lin_reg.fit(training)

In [8]:
model.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [9]:
model.intercept

0.14228558260358093

In [10]:
train_summary=model.summary

In [12]:
all_data=spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [13]:
train_data,test_data=all_data.randomSplit([0.7,0.3])

In [14]:
train_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 342|
|   mean|-0.15981918183456495|
| stddev|  10.269757610761973|
|    min| -26.805483428483072|
|    max|  27.111027963108548|
+-------+--------------------+



In [15]:
new_model=lin_reg.fit(train_data)

In [16]:
test_results=new_model.evaluate(test_data)

In [17]:
test_results.rootMeanSquaredError

10.4443876349525

In [18]:
unlabelled_data=test_data.select('features')

In [19]:
unlabelled_data.show(4)

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 4 rows



In [20]:
preds=new_model.transform(unlabelled_data)

In [21]:
preds.show(5)

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -0.4733636864199491|
|(10,[0,1,2,3,4,5,...|-0.36571034866809954|
|(10,[0,1,2,3,4,5,...|  -2.066163333358208|
|(10,[0,1,2,3,4,5,...| 0.43400805778943574|
|(10,[0,1,2,3,4,5,...|   1.401279375966562|
+--------------------+--------------------+
only showing top 5 rows

